In [50]:
#Question 1 Part a

from data_utils import load_dataset
import numpy as np
import math 
import scipy
from matplotlib import pyplot as plt
from data_utils import load_dataset

#ALVIN PANE 
#1004281118

def likelihood(x, y):
    likelihood = 1
    for i in range(len(x)):
        likelihood *= (sigmoid_b(x[i]) ** y[i]) * ((1 - sigmoid_b(x[i])) ** (1 - y[i]))
    return likelihood

def prior_LL(w, variance):
    prior = 1
    for i in range(len(w)):
        prior *= 1 / math.sqrt(2 * math.pi * variance) * math.exp(-(w[i] ** 2) / (2 * variance))
    return prior


def proposal_like(w, proposal_var, mean):
    proposal = 1
    for i in range(len(w)):
        proposal *= 1 / math.sqrt(2 * math.pi * proposal_var) * math.exp(-((mean[i] - w[i]) ** 2) / (2 * proposal_var))
    return proposal

def sigmoid(w,x):
    z=np.dot(w.transpose(),x)
    return 1 / (1 + np.exp(-z))

#log likelihood
def LL(w,estimates, actual):
    accum = 0
    for i in range(len(estimates)):
        accum += actual[i]*np.log(sigmoid(w,estimates[i])) + (1-actual[i])*np.log(1 - sigmoid(w,estimates[i]))
    return accum

# Question 1a ---------------------------------------------------------------

#Compute the Hessian for computing the Laplace Approximation.
def Hess(x_t,w,variance):
    #apply hessian function
    h_l=np.zeros((len(x_t[0]),len(x_t[0])))
    h_p=-1*np.identity(len(x_t[0]))/variance
    for i in x_t:
        h_l+=sigmoid(w,i)*(sigmoid(w,i)-1)*np.outer(i,i.transpose())
    ret = h_l + h_p
    return ret

def marginal_LL(x_train,w,var,y_train):
    dim=len(x_train[0])
    det=np.linalg.det(-1*Hess(x_train,w,var))
    return LL(w,x_train,y_train)-0.5*dim*(np.log(2*var*np.pi))-np.sum(np.square(w))/(2*var)+0.5*dim*np.log(2*np.pi)-0.5*np.log(det)

# Question 1b -------------------------------------------------------------

# HELPER FUNCTIONS
def acc_ratio(y_t, y_e):
    ar = (y_e == y_t).sum() / len(y_t)
    return ar

def matrix(x_data):
    X = np.ones((len(x_data), len(x_data[0]) + 1))
    X[:, 1:] = x_data
    return X


def sigmoid_b(z):
    return np.divide(1, np.add(1, np.exp(-1*z)))

def log_likelihood(x_prod, y_act):
    log_p = np.dot(y_act.T, np.log(sigmoid_b(x_prod))) + np.dot(np.subtract(1, y_act).T, np.log(np.subtract(1, sigmoid_b(x_prod))))
    return log_p

def r(x, y, w, prior_var, proposal_var, mean):
    return likelihood(x, y) * prior_LL(w, prior_var) / proposal_like(w, proposal_var, mean)


def proposal(mean, variance):
    return np.random.multivariate_normal(mean=mean, cov=np.eye(np.shape(mean)[0]) * variance)


def sample_w(sample_size, mean, variance):
    w = list()
    for i in range(sample_size):
        w.append(proposal(mean, variance))
    return w


def LL_2(y_pred, y):
    log_p = np.dot(y.T, np.log(y_pred)) + np.dot(np.subtract(1, y).T, np.log(np.subtract(1, y_pred)))
    return log_p


# --- Question 2 ---

def sampling(x_train, x_valid, x_test, y_train, y_valid, y_test, mean, sample_sizes):

    #init
    prior_var = 1 #given
    variances = [1, 2, 2.5, 5, 10]
    y_train = np.asarray(y_train, int)
    y_valid = np.asarray(y_valid, int)
    y_test = np.asarray(y_test, int)
    X_train = matrix(x_train)
    X_valid = matrix(x_valid)
    X_test = matrix(x_test)
    min_ll = 1000000000
    
    #set up proposal distribution
    for sample_size in sample_sizes:
        for proposal_variance in variances:

            v_predict = np.zeros(np.shape(y_valid))
            valid_dpred = np.zeros(np.shape(y_valid))

            # sample s number of weights
            w = sample_w(sample_size, mean, proposal_variance)

            # compute predictions over the whole validation set
            for d in range(len(X_valid)):

                # inner sample over j
                inner_sum = 0
                i=0
                while i<(sample_size):
                #for j in range(sample_size):
                    # compute and sum r(w_js)
                    inner_sum += r(np.dot(X_train, w[i]), y_train, w[i], prior_var, proposal_variance, mean)
                    i+=1
                    
                # outer sample over i
                out_sum = 0
                j=0
                while j<sample_size:
                #for i in range(sample_size):
                    # compute sigmoid prediction of test point
                    y_star = sigmoid_b(np.dot(X_valid[d], w[j]))
                    # add to prediction summation 
                    out_sum += y_star*r(np.dot(X_train, w[j]), y_train, w[j], prior_var, proposal_variance, mean)/inner_sum
                    j+=1
                    
                # classify
                v_predict[d] = out_sum
                if out_sum > 0.5:
                    valid_dpred[d] = 1
                elif out_sum < 0.5:
                    valid_dpred[d] = 0
                else:
                    valid_dpred[d] = -1

            valid_logl = -LL_2(v_predict, y_valid)
            if valid_logl < min_ll:
                min_ll = valid_logl
                min_acc = acc_ratio(valid_dpred, y_valid)
                optimal_var = proposal_variance
                optimal_samps = sample_size

    #update
    x_train = np.vstack((x_train, x_valid))
    X_train = matrix(x_train)
    y_train = np.vstack((y_train, y_valid))
    test_pred = np.zeros(np.shape(y_test))
    test_discrete_pred = np.zeros(np.shape(y_test))
    w = sample_w(optimal_samps, mean, optimal_var)

    #compute predictions on test set
    for d in range(len(X_test)):
        # inner sample over j
        r_sum = 0
        for j in range(optimal_samps):
            # compute and sum r(w_js)
            r_sum += r(np.dot(X_train, w[j]), y_train, w[j], prior_var, optimal_var, mean)

        # outer sample over i
        pred_sum = 0
        for i in range(optimal_samps):
            # compute sigmoid prediction of test point
            y_star = sigmoid_b(np.dot(X_test[d], w[i]))
            # add to prediction summation 
            pred_sum += y_star*r(np.dot(X_train, w[i]), y_train, w[i], prior_var, optimal_var, mean)/r_sum

        # make classification (discretized and continuous)
        prediction = pred_sum
        test_pred[d] = prediction
        if prediction > 0.5:
            test_discrete_pred[d] = 1
        elif prediction < 0.5:
            test_discrete_pred[d] = 0
        else:
            test_discrete_pred[d] = -1

    test_accuracy_ratio = acc_ratio(test_discrete_pred, y_test)
    test_log_likelihood = compute_log_likelihood(test_pred, y_test)

    # sample s number of weights
    w = sample_w(5000, mean, 1)

    # inner sample over j
    r_sum = 0
    for j in range(5000):
        # compute and sum r(w_js)
        r_sum += r(np.dot(X_train, w[j]), y_train, w[j], prior_var, 2, mean)

    # outer sample over i
    posterior = list()
    for i in range(5000):
        # add to prediction summation
        posterior.append(r(np.dot(X_train, w[i]), y_train, w[i], prior_var, 2, mean) / r_sum)

    plot_posterior(mean, 1, posterior, w)

    return -test_log_likelihood, test_accuracy_ratio, optimal_var, optimal_samps, min_ll, min_acc

def plot_posterior(mean, variance, posterior, w):
    # five components to each weight vector
    for i in range(5):
        weights = list()
        # extract specific component of all weights
        for j in range(len(w)):
            weights.append(w[j][i])

        # zip weights and posterior
        weights, posterior = zip(*sorted(zip(weights, posterior)))

        # set up gaussian
        z = np.polyfit(weights, posterior, 1)
        z = np.squeeze(z)
        p = np.poly1d(z)

        # plot
        w_all = np.arange(min(weights), max(weights), 0.001)
        q_w = scipy.stats.norm.pdf(w_all, mean[i], variance)
        plt.figure(i)
        plt.title("Posterior visualization: w(" + str(i) + "), var=" + str(variance))
        plt.xlabel("w[" + str(i) + "]")
        plt.ylabel("Probability")
        plt.plot(w_all, q_w, '-g', label="Proposal")
        plt.plot(weights, posterior, 'ob', label="Posterior")
        plt.plot(weights, p(weights),"r--")
        plt.legend(loc='upper left')
 
#MAIN

Q1a = False   #SET TRUE TO SHOW ANSWER

Q1b = True    #SET TRUE TO SHOW ANSWER

xtrain, xvalid, xtest, ytrain, yvalid, ytest=load_dataset('iris')

#merge training and valid datasets
xtrain=np.hstack((np.ones((len(xtrain),1)),xtrain))
xvalid=np.hstack((np.ones((len(xvalid),1)),xvalid))
xtest=np.hstack((np.ones((len(xtest),1)),xtest))
xtrain=np.vstack((xtrain,xvalid))
ytrain=1*ytrain[:,(1,)]
yvalid=1*yvalid[:,(1,)]
ytrain=np.vstack((ytrain,yvalid))
ytest=1*ytest[:,(1,)]
init_weights=np.zeros((len(xtrain[0]),1))

if __name__ == '__main__':
    all_datas = ['mauna_loa', 'rosenbrock', 'pumadyn32nm', 'iris', 'mnist_small']    

if Q1a:
    print('Question 1a----------------------------')
    print('For variance = 0.5,      ' + str(marginal_LL(xtrain,w,0.5,ytrain)))
    print('For variance = 1,        ' + str(marginal_LL(xtrain,w,1,ytrain)))
    print('For variance = 2,        ' + str(marginal_LL(xtrain,w,2,ytrain)))
    
if Q1b:
    print('Question 1b----------------------------')
    x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('iris')
    y_train, y_valid, y_test = y_train[:, (1,)], y_valid[:, (1,)], y_test[:, (1,)]


    # --- Question 2 ---

    MAP = [-0.87804, 0.29303, -1.23510, 0.67820, -0.89421] #values from part a)
    samples=[10, 25, 50, 100, 200,300]
    test_logl, test_accratio, prop_var, samplesize, valid_logl, valid_accratio = sampling(x_train, x_valid, x_test, y_train,y_valid,y_test, MAP, samples)
                                                                                                                                                           
    print('Proposal Distribution Variance: ' + str(prop_var))
    print('Sample Size: ' + str(samplesize))
    print('Validation Log-likelihood: ' + str(valid_logl))
    print('Validation Accuracy Ratio: ' + str(valid_accratio))
    print('Test Log-likelihood: ' + str(test_logl))
    print('Test Accuracy Ratio: ' + str(test_accratio))


Question 1b----------------------------
Q1b


KeyboardInterrupt: 